In [1]:
import pandas as pd
import numpy as np
import warnings

# Mengabaikan peringatan untuk kejelasan output
warnings.filterwarnings('ignore')

In [2]:
# --- 1. Fungsi Pemuatan Data (Load Data) ---
def load_data():
    """Memuat semua file CSV."""
    file_paths = {
        "train": "application_train.csv",
        "test": "application_test.csv",
        "bureau": "bureau.csv",
        "bureau_balance": "bureau_balance.csv",
        "pos_cash": "POS_CASH_balance.csv",
        "credit_card": "credit_card_balance.csv",
        "previous": "previous_application.csv",
        "installments": "installments_payments.csv"
        # HomeCredit_columns_description.csv tidak diperlukan untuk feature engineering
    }

    data = {}
    print("Memuat data...")
    for name, path in file_paths.items():
        try:
            data[name] = pd.read_csv(path)
            print(f"  ✅ '{name}' dimuat: {data[name].shape[0]} baris.")
        except FileNotFoundError:
            print(f"  ❌ ERROR: File '{path}' tidak ditemukan. Lewati.")
    return data

In [3]:
# --- 2. Fungsi Agregasi Bureau dan Bureau Balance ---

def bureau_and_balance_agg(bureau, bureau_balance):
    """
    Melakukan agregasi bertingkat:
    1. Agregasi bureau_balance per SK_ID_BUREAU.
    2. Gabungkan hasil ke bureau.
    3. Agregasi bureau per SK_ID_CURR.
    """
    print("\n[START] Feature Engineering: Bureau & Bureau Balance...")

    # 1. Agregasi Tingkat Rendah (bureau_balance -> SK_ID_BUREAU)
    # Agregasi Fitur Kategorikal (STATUS): Hitungan Frekuensi
    bb_counts = bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].value_counts(normalize=True).unstack(fill_value=0)
    bb_counts.columns = [f'BB_STATUS_{col}_RATIO' for col in bb_counts.columns]

    # Agregasi Fitur Numerik
    bb_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(
        BB_MONTHS_COUNT=('MONTHS_BALANCE', 'count'),
        BB_MONTHS_MEAN=('MONTHS_BALANCE', 'mean')
    )

    # Gabungkan hasil agregasi bb
    bb_final = pd.merge(bb_counts, bb_agg, left_index=True, right_index=True, how='outer').reset_index()

    # Gabungkan hasil agregasi bb ke tabel bureau
    bureau_merged = pd.merge(bureau, bb_final, on='SK_ID_BUREAU', how='left')

    # 2. Agregasi Tingkat Menengah (bureau_merged -> SK_ID_CURR)
    bureau_agg = bureau_merged.groupby('SK_ID_CURR').agg({
        # Fitur dari bureau
        'SK_ID_BUREAU': 'count', # Jumlah pinjaman sebelumnya
        'AMT_CREDIT_SUM': ['mean', 'max', 'sum'],
        'DAYS_CREDIT': ['mean', 'max'],
        'CREDIT_DAY_OVERDUE': ['mean', 'max'],
        # Fitur dari bureau_balance (setelah digabungkan)
        'BB_MONTHS_COUNT': ['mean', 'sum']
        # Anda dapat menambahkan kolom BB_STATUS_*_RATIO di sini juga jika diperlukan
    })

    # Meratakan kolom dan memberi nama dengan prefix
    bureau_agg.columns = [f'BUREAU_{"_".join(col).upper()}' for col in bureau_agg.columns]
    bureau_agg = bureau_agg.reset_index()

    print("[END] Feature Engineering: Bureau & Bureau Balance selesai.")
    return bureau_agg

In [4]:
# --- 3. Fungsi Utama: Main Execution ---

def run_feature_engineering_and_merge():
    """Mengelola pemuatan, agregasi, dan penggabungan akhir."""

    data = load_data()

    # Pastikan data utama ada
    if 'train' not in data or 'test' not in data:
        print("\n[GAGAL] Data aplikasi utama (train/test) tidak lengkap.")
        return

    # Ambil TARGET dari data train
    train_target = data['train']['TARGET']

    # Inisialisasi DataFrame akhir untuk train dan test
    df_train_final = data['train'].copy()
    df_test_final = data['test'].copy()

    # --- PROSES A: Agregasi Tabel Pendukung ---

    all_features = {}

    # A. Bureau & Bureau Balance
    if 'bureau' in data and 'bureau_balance' in data:
        bureau_features = bureau_and_balance_agg(data['bureau'], data['bureau_balance'])
        all_features['bureau'] = bureau_features

    # B. Ulangi proses untuk tabel lain di sini (misalnya Previous Application)
    # if 'previous' in data:
    #     prev_features = previous_application_agg(data['previous'])
    #     all_features['previous'] = prev_features

    # --- PROSES B: Penggabungan (Merge) Akhir ke Data Aplikasi ---

    print("\n[START] Penggabungan Fitur ke Train dan Test...")

    for name, features_df in all_features.items():
        # Gabung ke Data Train
        df_train_final = pd.merge(
            df_train_final,
            features_df,
            on='SK_ID_CURR',
            how='left',
            suffixes=('', f'_{name}_left') # Hindari konflik nama kolom
        )

        # Gabung ke Data Test
        df_test_final = pd.merge(
            df_test_final,
            features_df,
            on='SK_ID_CURR',
            how='left',
            suffixes=('', f'_{name}_right')
        )
        print(f"  ✅ Fitur dari '{name}' berhasil digabungkan.")

      # --- HASIL AKHIR ---

    print("\n=============================================")
    print("🎉 Proses Feature Engineering Selesai")
    print("=============================================")
    print(f"Dimensi Data Training Final: {df_train_final.shape}")
    print(f"Dimensi Data Testing Final: {df_test_final.shape}")

    print("\nHead Data Training (dengan fitur baru):")
    print(df_train_final.head())

    print("\nHead Data Testing (dengan fitur baru):")
    print(df_test_final.head())

    print("\nStatus Kolom TARGET di Data Final:")
    print(f"Train: {'TARGET' in df_train_final.columns}")
    print(f"Test: {'TARGET' in df_test_final.columns}")

    return df_train_final, df_test_final

# Eksekusi script utama
if __name__ == "__main__":
    df_train_ready, df_test_ready = run_feature_engineering_and_merge()

# Setelah skrip berjalan, variabel df_train_ready dan df_test_ready
# akan berisi DataFrame siap model dengan semua fitur yang digabungkan.

Memuat data...
  ✅ 'train' dimuat: 307511 baris.
  ✅ 'test' dimuat: 48744 baris.
  ✅ 'bureau' dimuat: 1716428 baris.
  ✅ 'bureau_balance' dimuat: 27299925 baris.
  ✅ 'pos_cash' dimuat: 10001358 baris.
  ✅ 'credit_card' dimuat: 3840312 baris.
  ✅ 'previous' dimuat: 1670214 baris.
  ✅ 'installments' dimuat: 13605401 baris.

[START] Feature Engineering: Bureau & Bureau Balance...
[END] Feature Engineering: Bureau & Bureau Balance selesai.

[START] Penggabungan Fitur ke Train dan Test...
  ✅ Fitur dari 'bureau' berhasil digabungkan.

🎉 Proses Feature Engineering Selesai
Dimensi Data Training Final: (307511, 132)
Dimensi Data Testing Final: (48744, 131)

Head Data Training (dengan fitur baru):
   SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0     

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

In [12]:
# =========================================================================
# ⚠️ PENTING: PENGATURAN DATA AWAL (Sesuaikan Kebutuhan)
# =========================================================================
try:
    # Coba muat data asli dan lakukan Feature Engineering minimal (Simulasi)
    # CATATAN: Jika Anda menjalankan skrip lengkap dari awal, hapus .sample()
    data = load_data()

    # 1. Ambil Data Aplikasi (Full, tanpa .sample() jika ingin menggunakan semua data)
    df_train = data['train'].copy()
    df_test = data['test'].copy()

    # 2. Gabungkan fitur Bureau ke data aplikasi (menggunakan fungsi yang sudah didefinisikan)
    bureau_features = bureau_and_balance_agg(data['bureau'], data['bureau_balance'])

    # Merge fitur yang diagregasi ke train dan test
    df_train_ready = pd.merge(df_train, bureau_features, on='SK_ID_CURR', how='left')
    df_test_ready = pd.merge(df_test, bureau_features, on='SK_ID_CURR', how='left')

    # Pastikan TARGET dihapus dari set test (jika ada karena kesalahan)
    if 'TARGET' in df_test_ready.columns:
        df_test_ready = df_test_ready.drop('TARGET', axis=1)

except Exception as e:
    print(f"\n[ERROR: PASTIKAN FILE DATA ADA] Gagal memuat/memproses data: {e}")
    # Jika gagal, hentikan eksekusi
    exit()

print("\n--- Persiapan Data untuk Pemodelan ---")

# 1. Pemisahan Final Fitur (X) dan Target (y)
print("Memisahkan ID, Target, dan Fitur...")

# Simpan ID Pelanggan dari Test Set untuk file submission
test_ids = df_test_ready['SK_ID_CURR']

# Hapus ID Pelanggan dan TARGET dari fitur training (X_train)
X_train_full = df_train_ready.drop(['TARGET', 'SK_ID_CURR'], axis=1)
y_train = df_train_ready['TARGET']

# Hapus ID Pelanggan dari fitur testing (X_test)
X_test_full = df_test_ready.drop('SK_ID_CURR', axis=1)

# 2. Identifikasi Tipe Kolom
print("Mengidentifikasi Tipe Kolom...")

# Ambil kolom yang ada di X_train_full
numerical_cols = X_train_full.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X_train_full.select_dtypes(include='object').columns.tolist()

# Filter agar hanya kolom yang ada di kedua set yang digunakan
numerical_cols = [col for col in numerical_cols if col in X_test_full.columns]
categorical_cols = [col for col in categorical_cols if col in X_test_full.columns]


Memuat data...
  ✅ 'train' dimuat: 307511 baris.
  ✅ 'test' dimuat: 48744 baris.
  ✅ 'bureau' dimuat: 1716428 baris.
  ✅ 'bureau_balance' dimuat: 27299925 baris.
  ✅ 'pos_cash' dimuat: 10001358 baris.
  ✅ 'credit_card' dimuat: 3840312 baris.
  ✅ 'previous' dimuat: 1670214 baris.
  ✅ 'installments' dimuat: 13605401 baris.

[START] Feature Engineering: Bureau & Bureau Balance...
[END] Feature Engineering: Bureau & Bureau Balance selesai.

--- Persiapan Data untuk Pemodelan ---
Memisahkan ID, Target, dan Fitur...
Mengidentifikasi Tipe Kolom...


In [13]:
# 2. Identifikasi Tipe Kolom
# Ambil kolom yang ada di X_train_full dan X_test_full
numerical_cols = X_train_full.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X_train_full.select_dtypes(include='object').columns.tolist()

# Karena feature engineering yang rumit bisa menghasilkan kolom yang tidak valid,
# kita hanya akan menggunakan kolom numerik dan beberapa kategorikal untuk penyederhanaan.
numerical_cols = [col for col in numerical_cols if col in X_test_full.columns]
categorical_cols = [col for col in categorical_cols if col in X_test_full.columns]

In [14]:
# --- 3. Pipelining Pra-pemrosesan ---

# Pra-pemrosesan untuk kolom numerik: Imputasi dengan Median, lalu Scaling
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pra-pemrosesan untuk kolom kategorikal: Imputasi dengan nilai paling sering, lalu One-Hot Encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # 'ignore' agar fitur baru di test set diabaikan
])

# ColumnTransformer untuk menerapkan transformasi berbeda ke tipe kolom berbeda
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='drop' # Hapus kolom yang tidak termasuk dalam num/cat (misalnya kolom waktu yang sudah diolah)
)

In [15]:
# --- 4. Membangun dan Melatih Model Regresi Logistik ---

# Model (Pipeline penuh: Preprocessor -> Model)
# C parameter adalah Regularization inverse strength (semakin kecil C, semakin kuat regularisasinya)
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(C=0.0001, solver='liblinear', random_state=42, class_weight='balanced'))
])

print("\n[START] Melatih Model Regresi Logistik...")
# Melatih model
model.fit(X_train_full, y_train)
print("[END] Pelatihan Model Selesai.")


[START] Melatih Model Regresi Logistik...
[END] Pelatihan Model Selesai.


In [16]:
# --- 5. Evaluasi dan Prediksi ---

# A. Evaluasi pada Training Set (Untuk melihat baseline kinerja)
y_train_pred_proba = model.predict_proba(X_train_full)[:, 1]
train_auc = roc_auc_score(y_train, y_train_pred_proba)

print(f"\n=============================================")
print(f"🎉 Kinerja Model (Training Set - AUC): {train_auc:.4f}")
print("=============================================")

# B. Prediksi pada Testing Set
# Regresi Logistik biasanya digunakan untuk memprediksi probabilitas (predict_proba)
test_predictions_proba = model.predict_proba(X_test_full)[:, 1]


🎉 Kinerja Model (Training Set - AUC): 0.7430


In [11]:
# --- 6. Pembuatan Submission File (Output) ---

submission = pd.DataFrame({
    'SK_ID_CURR': test_ids,
    'TARGET': test_predictions_proba
})

# Simpan hasil prediksi ke file CSV
submission.to_csv('logistic_regression_submission.csv', index=False)

print("\n✅ Prediksi Data Test Selesai dan disimpan ke 'logistic_regression_submission.csv'")
print("Head dari file submission:")
print(submission.head())


✅ Prediksi Data Test Selesai dan disimpan ke 'logistic_regression_submission.csv'
Head dari file submission:
   SK_ID_CURR    TARGET
0      208550  0.455112
1      173779  0.295916
2      365820  0.440610
3      144092  0.474715
4      291599  0.496877
